<a href="https://colab.research.google.com/github/imyoungchae/Tensorflow_study/blob/main/tensorflow10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence 데이터 다루기

# Import

In [1]:
import urllib
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

# 아래 2줄 코드는 넣지 말아 주세요!!!
url = 'https://www.dropbox.com/s/eduk281didil1km/Weekly_U.S.Diesel_Retail_Prices.csv?dl=1'
urllib.request.urlretrieve(url, 'Weekly_U.S.Diesel_Retail_Prices.csv')

('Weekly_U.S.Diesel_Retail_Prices.csv',
 <http.client.HTTPMessage at 0x7facf0407fd0>)

#Normalization

In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# Windowed Dataset

In [4]:
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

# Dataset Load

In [5]:
df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv', infer_datetime_format=True, index_col='Week of', header=0)
df.head(20)

,Weekly U.S. No 2 Diesel Retail Prices Dollars per Gallon
Week of,
1994-03-21,1.106
1994-03-28,1.107
1994-04-04,1.109
1994-04-11,1.108
1994-04-18,1.105
1994-04-25,1.106
1994-05-02,1.104
1994-05-09,1.101
1994-05-16,1.099


In [6]:
# 특성 정의
N_FEATURES = len(df.columns)
N_FEATURES

1

# 정규화 및 데이터 분할

In [7]:
# 정규화 코드
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# 데이터 분할
SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [8]:
BATCH_SIZE = 32  # 배치사이즈
N_PAST = 10      # 과거 데이터 (X)
N_FUTURE = 10    # 미래 데이터 (Y)
SHIFT = 1        # SHIFT

# Train/Validation Set

In [9]:
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)

valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)

# 모델 구성

In [10]:
model = tf.keras.models.Sequential([
    Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(32, return_sequences=True)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(N_FEATURES)
])

# CheckPoint 생성

In [11]:
checkpoint_path = 'model/my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_mae',
                             verbose=1)

# Compile

In [12]:
optimizer = tf.keras.optimizers.Adam(0.0001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])

# Fit

In [13]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=[checkpoint])

Epoch 1/100
     35/Unknown - 9s 22ms/step - loss: 0.0530 - mae: 0.2522
Epoch 1: val_mae improved from inf to 0.32522, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 12s 96ms/step - loss: 0.0530 - mae: 0.2522 - val_loss: 0.0549 - val_mae: 0.3252
Epoch 2/100
35/35 [==============================] - ETA: 0s - loss: 0.0375 - mae: 0.2012
Epoch 2: val_mae improved from 0.32522 to 0.24983, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 27ms/step - loss: 0.0375 - mae: 0.2012 - val_loss: 0.0333 - val_mae: 0.2498
Epoch 3/100
33/35 [===========================>..] - ETA: 0s - loss: 0.0210 - mae: 0.1494
Epoch 3: val_mae improved from 0.24983 to 0.15003, saving model to model/my_checkpoint.ckpt
35/35 [==============================] - 1s 27ms/step - loss: 0.0208 - mae: 0.1498 - val_loss: 0.0136 - val_mae: 0.1500
Epoch 4/100
35/35 [==============================] - ETA: 0s - loss: 0.0106 - mae: 0.1175
Epoch 4: val_mae improved

In [14]:
model.load_weights(checkpoint_path)